In [1]:
%%time
import numpy as np
import pandas as pd
import os
os.chdir('E:\cstz_pci')

Wall time: 659 ms


In [2]:
tplq=pd.read_csv('查询EUTRAN同频邻区关系.csv',encoding="gbk",usecols=['NAME','本地小区标识','基站标识','小区标识'],dtype={'本地小区标识':str,'基站标识':str,'小区标识':str})

In [3]:
cell_info=pd.read_excel('小区信息.xlsx',sheet_name='小区基础信息',dtype={'物理小区标识':str})

In [4]:
pci=pd.read_excel('pci.xlsx')

In [5]:
wb=pd.read_csv('查询EUTRAN外部小区.csv',encoding="gbk",usecols=['NAME','基站标识','小区标识','物理小区标识'],dtype={'基站标识':str,'小区标识':str,'物理小区标识':str})

In [6]:
wb['基站标识_小区标识']=wb['基站标识'].map(str)+"_"+wb['小区标识'].map(str)

In [7]:
tplq['NAME_本地小区标识']=tplq['NAME']+"_"+tplq['本地小区标识']

In [8]:
tplq['eNodeB标识_小区标识']=tplq['基站标识']+"_"+tplq['小区标识']

In [9]:
tplq=pd.merge(tplq,cell_info,how='left',on='eNodeB标识_小区标识')

In [10]:
tplq=tplq[['NAME_x','本地小区标识_x','基站标识','小区标识_x','NAME_本地小区标识_x','eNodeB标识_小区标识','物理小区标识']]

In [11]:
pci_ori=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80','81','82','83','84','85','86','87','88','89','90','91','92','93','94','95','96','97','98','99','100','101','102','103','104','105','106','107','108','109','110','111','112','113','114','115','116','117','118','119','120','121','122','123','124','125','126','127','128','129','130','131','132','133','134','135','136','137','138','139','140','141','142','143','144','145','146','147','148','149','150','151','152','153','154','155','156','157','158','159','160','161','162','163','164','165','166','167','168','169','170','171','172','173','174','175','176','177','178','179','180','181','182','183','184','185','186','187','188','189','190','191','192','193','194','195','196','197','198','199','200','201','202','203','204','205','206','207','208','209','210','211','212','213','214','215','216','217','218','219','220','221','222','223','224','225','226','227','228','229','230','231','232','233','234','235','236','237','238','239','240','241','242','243','244','245','246','247','248','249','250','251','252','253','254','255','256','257','258','259','260','261','262','263','264','265','266','267','268','269','270','271','272','273','274','275','276','277','278','279','280','281','282','283','284','285','286','287','288','289','290','291','292','293','294','295','296','297','298','299','300','301','302','303','304','305','306','307','308','309','310','311','312','313','314','315','316','317','318','319','320','321','322','323','324','325','326','327','328','329','330','331','332','333','334','335','336','337','338','339','340','341','342','343','344','345','346','347','348','349','350','351','352','353','354','355','356','357','358','359','360','361','362','363','364','365','366','367','368','369','370','371','372','373','374','375','376','377','378','379','380','381','382','383','384','385','386','387','388','389','390','391','392','393','394','395','396','397','398','399','400','401','402','403','404','405','406','407','408','409','410','411','412','413','414','415','416','417','418','419','420','421','422','423','424','425','426','427','428','429','430','431','432','433','434','435','436','437','438','439','440','441','442','443','444','445','446','447','448','449','450','451','452','453','454','455','456','457','458','459','460','461','462','463','464','465','466','467','468','469','470','471','472','473','474','475','476','477','478','479','480','481','482','483','484','485','486','487','488','489','490','491','492','493','494','495','496','497','498','499','500','501','502','503']

In [12]:
tplq_pci=list(tplq['物理小区标识'].unique())

In [13]:
# 取差集
pci_f=list(set(tplq_pci)^set(pci_ori))

In [14]:
# 将列表装换成DataFrame
pci_f=pd.DataFrame(pci_f)

In [15]:
# 删除空值
pci_f=pci_f.dropna()
# while None in pci_f:
#     pci_f.remove(None)

In [16]:
# 删除索引，以便合并
pci.reset_index(drop=True,inplace=True)
pci_f.reset_index(drop=True,inplace=True)

In [17]:
# 生成过渡的pci数据
pci=pd.concat([pci,pci_f],join='inner',axis=1)

In [18]:
pci.rename(columns={0:'过渡pci'},inplace=True)

In [19]:
pci[['物理小区标识_NEW','过渡pci']]=pci[['物理小区标识_NEW','过渡pci']].astype(str)

In [20]:
wb=pd.merge(wb,pci,how='left',left_on='基站标识_小区标识',right_on='eNodeB标识_小区标识')

In [21]:
# 定义pci修改函数
def pci_1(c,d):
    return f"""DEA CELL:LOCALCELLID={d};{{{c}}}"""
def pci_2(b,c,d):
    return f"""MOD CELL:LOCALCELLID={d},PHYCELLID={b};{{{c}}}"""
def pci_3(a,c,d):
    return f"""MOD CELL:LOCALCELLID={d},PHYCELLID={a};{{{c}}}"""
def pci_4(c,d):
    return f"""ACT CELL:LOCALCELLID={d};{{{c}}}"""
def heca1(c): 
    return f"""DSP CELL:;{{{c}}}"""
def heca2(c):
    return f"""LST CELL:;{{{c}}}"""
def heca3(c):
    return f"""LST ALMAF:;{{{c}}}"""
def wb_1(e,f,g,h):
    return f"""MOD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={f},CELLID={g},PHYCELLID={h};{{{e}}}"""
 

In [22]:
pci['mml1']=pci.apply(lambda x : pci_1(x['NAME'],x['本地小区标识']),axis=1)
pci['mml2']=pci.apply(lambda x : pci_2(x['过渡pci'],x['NAME'],x['本地小区标识']),axis=1)
pci['mml3']=pci.apply(lambda x : pci_3(x['物理小区标识_NEW'],x['NAME'],x['本地小区标识']),axis=1)
pci['mml4']=pci.apply(lambda x : pci_4(x['NAME'],x['本地小区标识']),axis=1)
pci['mml5']=pci.apply(lambda x : heca1(x['NAME']),axis=1)
pci['mml6']=pci.apply(lambda x : heca2(x['NAME']),axis=1)
pci['mml7']=pci.apply(lambda x : heca3(x['NAME']),axis=1)
wb['mml']=wb.apply(lambda y : wb_1(y['NAME_x'],y['基站标识'],y['小区标识_x'],y['物理小区标识_NEW']),axis=1)

In [23]:
tzpci=pci[['mml1','mml2','mml3','mml4']]

In [24]:
# 将列文件数据转换成行
tzpci=tzpci.unstack(level=-1)

In [25]:
tzpci=pd.DataFrame(tzpci)

In [26]:
tzpci.rename(columns={0:'mml'},inplace=True)

In [27]:
tzpci.to_csv('1_tzpci.txt',index=False,header=False,encoding='gbk',sep='"')

In [28]:
heca=pci[['mml5','mml6','mml7']]

In [29]:
heca.drop_duplicates(inplace=True)

c:\program files (x86)\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
heca=heca.unstack(level=-1)

In [31]:
heca=pd.DataFrame(heca)

In [32]:
heca.rename(columns={0:'mml'},inplace=True)

In [33]:
heca.to_csv('2_heca.txt',index=False,header=False,encoding='gbk',sep='"')

In [34]:
# 筛选出不含nan的数据
wb_1=wb[~wb['mml'].str.contains('nan')]

In [35]:
mml_wb=wb_1[['mml']]


In [36]:
mml_wb.to_csv('3_tzwb.txt',index=False,header=False,encoding='gbk',doublequote=False,quoting=3,escapechar="?",sep='?')

In [37]:
writer = pd.ExcelWriter("调整pci脚本 1-3.xlsx")
# 设置不导出索引和将不同的Dateframe 写入多个excel的sheet中
pci.to_excel(writer,'第1步调整pci',index=None)
# wb.to_excel(writer,'第2步调整外部',index=None)
writer.save()
print("pci调整脚本已导出成功")

pci调整脚本已导出成功
